In [71]:
# Imports
import numpy as np
import pandas as pd
import ipaddress
from pandas.tseries.holiday import USFederalHolidayCalendar
from sklearn.preprocessing import OneHotEncoder

# Training libs
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [72]:
# Feature groups
anomaly_detection_input_features = [
    "account_id",                 # Unique identifier for the user account
    "transaction_id",             # Unique identifier for the transaction (dropped in training)

    "date",                       # (String) Transaction date (YYYY-MM-DD)
    "hour_of_day",                # (Integer) Hour of day when tx occurred (0–23)
    "minute_of_hour",             # (Integer) Minute of the hour when tx occurred (0–59)
    "day_of_week",                # (Integer) Day of week (0=Mon ... 6=Sun)
    "is_weekend",                 # (Boolean) True if tx on Saturday or Sunday
    "is_holiday",                 # (Boolean) True if tx date is a public holiday
    "days_since_last_holiday",    # (Integer) Days since the most recent public holiday

    "amount",                     # (Float) Transaction amount
    "amount_to_avg_ratio",        # (Float) amount / rolling_mean_3mo_amount
    "amount_percentile",          # (Float) Percentile rank in user’s historical amounts
    "transaction_type",           # (String) Credit vs Debit
    "transaction_duration",       # (Integer) Time in seconds from initiation to settlement
    "is_recurring",               # (Boolean) Recurring (True) vs one-off (False)
    
    "new_payee_flag",             # (Boolean) True if merchant is new for user
    "avg_unique_merchants_7d",    # (Float) Avg. number of new merchants in 7-day window

    "rolling_mean_3mo_amount",    # (Float) 3-month trailing mean transaction amount
    "rolling_std_3mo_amount",     # (Float) 3-month trailing std deviation of amounts
    "time_since_last_tx_secs",    # (Integer) Seconds since previous transaction
    "tx_count_last_1h",           # (Integer) Number of transactions in the last 1 hour
    "tx_count_last_7d",           # (Integer) Number of transactions in the last 7 days
    "tx_sum_last_24h",            # (Float) Sum of amounts in the last 24 hours

    "location",                   # (String) City or ZIP code of transaction
    "is_new_device",              # (Boolean) True if this device fingerprint is new for user
    "device_age_days",            # (Integer) Days since device first seen
    "login_attempt_count",        # (Integer) Number of login attempts in the last 24 hours
    "age",                        # (Integer) User’s age in years

    # NEW STUFF TO PROCESS
    "account_balance",            # (Float) User’s account balance at the time of transaction
    "channel",                    # (String) Channel through which the transaction was made (e.g., ATM, online, etc.)
    "customer_occupation",        # (String) User’s occupation (e.g., engineer, teacher, etc.)
    "ip_address",                 # (String) User’s IP address 

    

    # # Removed Features
    # "category",                   # (String) Transaction category (e.g. "utilities", "salary")
    # "is_foreign_transaction",     # (Boolean) True if currency or country differs from user's home region
    # "merchant_name",              # (String) Vendor or payee name
    # "merchant_category",          # (String) High-level merchant category (e.g., food, utilities)
    # "browser",                    # (String) Browser fingerprint
    # "os",                         # (String) Operating system of the device
    # "ip_blacklist_flag",          # (Boolean) True if IP found on fraud blacklist
    # "vpn_detected",               # (Boolean) True if IP linked to known VPN or proxy
    # "account_age_days",           # (Integer) Days since account or card was opened
    # "past_chargeback_count",      # (Integer) Number of past chargebacks for the user
    # "user_segment",               # (String) Segment based on spending behavior (e.g., low, medium, high)
    # "mismatch_billing_shipping",  # (Boolean) True if billing address not same as shipping address
]


anomaly_detection_output_features = [
    # Output features
    "is_anomaly", # (Boolean): Flagged by the model as anomalous
    "anomaly_confidence", # (Float): Model’s confidence (0–1) in that flag
]

In [73]:
# Load the dataset with date parsing
df = pd.read_csv("assets/transactions.csv", parse_dates=['TransactionDate', 'PreviousTransactionDate'])

In [74]:
# Rename columns to match the feature list
rename_map = {
    'TransactionID': 'transaction_id',
    'AccountID': 'account_id',
    'TransactionAmount': 'amount',
    'TransactionDate': 'date',
    'TransactionType': 'transaction_type',
    'Location': 'location',
    'DeviceID': 'device_id',
    'IP Address': 'ip_address',
    'MerchantID': 'merchant_id',
    'Channel': 'channel',
    'CustomerAge': 'age',
    'CustomerOccupation': 'customer_occupation',
    'TransactionDuration': 'transaction_duration',
    'LoginAttempts': 'login_attempt_count',
    'AccountBalance': 'account_balance',
    'PreviousTransactionDate': 'previous_transaction_date'
}

df = df.rename(columns=rename_map)

# Sort data
df = df.sort_values(["account_id","date"])

In [75]:
# Preprocess features
# Extract date components
df["hour_of_day"]    = df["date"].dt.hour
df["minute_of_hour"] = df["date"].dt.minute
df["day_of_week"]    = df["date"].dt.dayofweek
df["is_weekend"]     = df["day_of_week"] >= 5

# Holidays
years = df["date"].dt.year.unique()
hols = USFederalHolidayCalendar().holidays(
    start=f"{years.min()-1}-01-01",
    end=  f"{years.max()+1}-12-31"
)
df["is_holiday"] = df["date"].dt.normalize().isin(hols)

def days_since_last(x):
    prev = hols[hols < x.normalize()]
    return (x.normalize() - prev.max()).days if len(prev) else (x.normalize() - hols.min()).days

df["days_since_last_holiday"] = df["date"].apply(days_since_last)

# Time since last transaction
df["time_since_last_tx_secs"] = (
    df.groupby("account_id")["date"]
      .diff()
      .dt.total_seconds()
      .fillna(0)
      .astype(int)
)

# Rolling statistics (per account)
r90 = df.groupby("account_id")\
        .rolling("90d", on="date", min_periods=1)["amount"]
df["rolling_mean_3mo_amount"] = r90.mean().reset_index(level=0, drop=True)
df["rolling_std_3mo_amount"]  = r90.std().reset_index(level=0, drop=True)

# Fill NaN values
df[["rolling_mean_3mo_amount","rolling_std_3mo_amount"]] = (
    df[["rolling_mean_3mo_amount","rolling_std_3mo_amount"]].fillna(0)
)

# Velocity features
w1h_exc  = df.groupby("account_id").rolling("1h",  on="date", closed="left", min_periods=0)["amount"]
w7d_exc  = df.groupby("account_id").rolling("7d",  on="date", closed="left", min_periods=0)["amount"]
w24h_exc = df.groupby("account_id").rolling("24h", on="date", closed="left", min_periods=0)["amount"]

df["tx_count_last_1h"] = w1h_exc.count().reset_index(level=0, drop=True)
df["tx_count_last_7d"] = w7d_exc.count().reset_index(level=0, drop=True)
df["tx_sum_last_24h"] = w24h_exc.sum().reset_index(level=0, drop=True)

# Fill NaN values
df[["tx_count_last_1h","tx_count_last_7d","tx_sum_last_24h"]] = (
    df[["tx_count_last_1h","tx_count_last_7d","tx_sum_last_24h"]]
    .fillna(0)
)
df[["tx_count_last_1h","tx_count_last_7d"]] = df[["tx_count_last_1h","tx_count_last_7d"]].astype(int)


# Derived ratios & percentiles
df["amount_to_avg_ratio"] = (
    df["amount"] / df["rolling_mean_3mo_amount"].replace(0, np.nan)
).fillna(1)
df["amount_percentile"] = df.groupby("account_id")["amount"].rank(pct=True)

# Recurrence & novelty
df["is_recurring"]   = df.duplicated(
    subset=["account_id","merchant_id","amount"], keep=False
)
df["new_payee_flag"] = df.groupby(
    ["account_id","merchant_id"]
).cumcount() == 0

# Merchant novelty
def compute_avg_unique_merchants_7d(group):
    merchants = group['merchant_id']
    dates = group['date']
    vals = []
    for current_time in dates:
        window_start = current_time - pd.Timedelta(days=7)
        mask = (dates >= window_start) & (dates <= current_time)
        vals.append(merchants[mask].nunique() / 7)
    return pd.Series(vals, index=group.index)

df = df.sort_values(['account_id','date'])
df['avg_unique_merchants_7d'] = (
    df.groupby('account_id')
      .apply(compute_avg_unique_merchants_7d)
      .reset_index(level=0, drop=True)
)

# Device context
df['is_new_device']   = df.groupby(
    ['account_id', 'device_id']
).cumcount() == 0
df['device_age_days'] = (
    df['date'] - df.groupby('device_id')['date'].transform('min')
).dt.days

/var/folders/gd/d2vxxv2x60j28jw85n3wg5dw0000gn/T/ipykernel_9993/2141539869.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_avg_unique_merchants_7d)


In [76]:
# Final Cleanup
final_features = [c for c in anomaly_detection_input_features if c in df.columns]
df = df[final_features]

df.head()

,account_id,transaction_id,date,hour_of_day,minute_of_hour,day_of_week,is_weekend,is_holiday,days_since_last_holiday,amount,...,tx_sum_last_24h,location,is_new_device,device_age_days,login_attempt_count,age,account_balance,channel,customer_occupation,ip_address
1312,AC00001,TX001313,2023-09-15 17:00:20,17,0,4,False,False,11,47.79,...,0.0,Denver,True,186,1,25,1649.92,Branch,Student,59.12.96.11
2016,AC00001,TX002017,2023-11-14 16:56:34,16,56,1,False,False,4,212.97,...,0.0,Atlanta,True,0,1,59,4180.40,Online,Engineer,45.241.13.208
2120,AC00002,TX002121,2023-01-10 16:00:32,16,0,1,False,False,8,476.99,...,0.0,San Diego,True,0,1,23,1154.48,Online,Student,113.137.153.101
20,AC00002,TX000021,2023-02-28 16:36:58,16,36,1,False,False,8,59.32,...,0.0,Los Angeles,True,0,1,71,5750.89,Branch,Retired,116.44.12.250
1476,AC00002,TX001477,2023-05-05 16:35:44,16,35,4,False,False,74,12.62,...,0.0,El Paso,True,66,1,33,6420.47,Branch,Doctor,93.160.83.196


In [77]:
# output the cleaned dataframe to a new CSV file
df.to_csv("assets/transactions_cleaned.csv", index=False)

In [78]:
# Convert categorical features to numerical
# Load preprocessed feature CSV
df = pd.read_csv("assets/transactions_cleaned.csv", parse_dates=["date"])

# Drop non-features
cols_to_drop = ["account_id", "transaction_id", "date"]
df_model = df.drop(columns=cols_to_drop)

# One hot encoding for categorical features
# One-hot encode "location" column and join into df_model
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
loc_mat = ohe.fit_transform(df_model[["location"]])
loc_cols = [f"loc_{c}" for c in ohe.categories_[0]]
df_loc = pd.DataFrame(loc_mat, columns=loc_cols, index=df_model.index)

df_model = pd.concat([
    df_model.drop(columns=["location"]), # Drop original location column
    df_loc
], axis=1)

# One-hot encode "channel" column and join into df_model
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
channel_mat = ohe.fit_transform(df_model[["channel"]])
channel_cols = [f"channel_{c}" for c in ohe.categories_[0]]
df_channel = pd.DataFrame(channel_mat, columns=channel_cols, index=df_model.index)

df_model = pd.concat([
    df_model.drop(columns=["channel"]), # Drop original channel column
    df_channel
], axis=1)

# One-hot encode "customer_occupation" column and join into df_model
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
occupation_mat = ohe.fit_transform(df_model[["customer_occupation"]])
occupation_cols = [f"occupation_{c}" for c in ohe.categories_[0]]
df_occupation = pd.DataFrame(occupation_mat, columns=occupation_cols, index=df_model.index)

df_model = pd.concat([
    df_model.drop(columns=["customer_occupation"]), # Drop original occupation column
    df_occupation
], axis=1)

# Other categorical columns
# Convert any boolean columns to 0/1
bool_cols = df_model.select_dtypes(include="bool").columns
for c in bool_cols:
    df_model[c] = df_model[c].astype(int)

# Convert credit/debit to binary
df_model['transaction_type'] = df_model['transaction_type'].map({
    'Credit': 1,
    'Debit': 0
})

# Convert IP address to numerical format
df_model['ip_address'] = df_model['ip_address'].apply(
    lambda x: int(ipaddress.ip_address(x))
)

In [79]:
# Scaling and Training
# Prepare numeric matrix and scale
X = df_model.astype(float)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train IsolationForest
iso = IsolationForest(
    n_estimators=500,
    max_samples=1.0,      
    max_features=1.0,
    bootstrap=True,
    contamination=0.05,
    n_jobs=-1,
    random_state=42,
    verbose=0,
)
iso.fit(X_scaled)

# Predict & score
labels     = iso.predict(X_scaled)         # 1 = normal, -1 = anomaly
raw_scores = iso.score_samples(X_scaled)   # higher = more normal

df["is_anomaly"] = labels == -1

inv = -raw_scores
df["anomaly_confidence"] = (inv - inv.min()) / (inv.max() - inv.min())

In [80]:
# Save/print results
important_head_cols = [
    "transaction_id", "amount", 
    "is_anomaly", "anomaly_confidence"
]
print(df[important_head_cols])
df.to_csv("assets/transactions_with_anomalies.csv", index=False)

     transaction_id  amount  is_anomaly  anomaly_confidence
0          TX001313   47.79       False            0.195565
1          TX002017  212.97       False            0.187227
2          TX002121  476.99       False            0.150111
3          TX000021   59.32       False            0.165020
4          TX001477   12.62       False            0.286291
...             ...     ...         ...                 ...
2507       TX001943  837.29       False            0.246154
2508       TX002055  156.86       False            0.124883
2509       TX001716  225.93       False            0.204888
2510       TX002375  129.62       False            0.292553
2511       TX002350  134.55       False            0.391922

[2512 rows x 4 columns]
